In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import gpflow
import gpflow.multioutput.kernels as mk
import gpflow.multioutput.features as mf
from MixtureSVGP import MixtureSVGP, generate_updates
import pickle
import sys
from actions import train_mixsvgp
from Assignments import Assignments, DPAssignments, GlobalAssignments

In [6]:
np.random.seed(0)

K = 2
L = 20
global_trajectories = False
save_path = './output/K2_L10'

n_iter = 200
minibatch_size = 5000


###############
# Make models #
###############
x, y, X, Y, weight_idx = pickle.load(
    open('data/ipsc_data_pickle', 'rb'))

assert(np.allclose(y[~np.isnan(y)] - Y.flatten(), 0))

T, N, G = y.shape
num_genes = G

x = x[:, :, :num_genes]
y = y[:, :, :num_genes]
mask = ~np.isnan(y)

Y = y[mask][:, None]
X = x[mask][:, None]
weight_idx = np.tile(np.arange(N*G).reshape(N, G)[None], (T, 1, 1))[mask]


# gp objects
if global_trajectories:
    num_clusters = K * L + L
else:
    num_clusters = K * L

kernel = mk.SharedIndependentMok(gpflow.kernels.RBF(1), num_clusters)
feature = mf.SharedIndependentMof(
    gpflow.features.InducingPoints(np.arange(T).astype(
        np.float64).reshape(-1, 1)))
likelihood = gpflow.likelihoods.Gaussian()

# model -- for hyperparameter learning
with gpflow.defer_build():
    m = MixtureSVGP(X, Y, weight_idx,
                    kern=kernel,
                    num_latent=num_clusters, num_data=X.shape[0],
                    likelihood=likelihood,
                    feat=feature, minibatch_size=minibatch_size)

    # only train hyperparams
    for param in m.parameters:
        param.trainable = False

# mean model -- for trajectory learning
# data are dummies, they'll be swapped out, just make correct shape
XB = np.arange(np.unique(X).size)[:, None].astype(np.float64)
YB = np.random.random([XB.shape[0], num_clusters]).astype(np.float64)

# put in model, weight_idx just arange because each observation unique
with gpflow.defer_build():
    m_bar = MixtureSVGP(XB, YB, np.arange(XB.size),
                        kern=kernel,
                        num_latent=num_clusters, num_data=XB.shape[0],
                        likelihood=likelihood,
                        feat=feature, minibatch_size=None)

    # fix all parameters except inducing outputs
    for param in m_bar.parameters:
        param.trainable = False

    m_bar.q_mu.trainable = True
    m_bar.q_sqrt.trainable = True

m.q_mu = m_bar.q_mu
m.q_sqrt = m_bar.q_sqrt

# compile models
m.compile()
m_bar.compile()

compute_weights, update_assignments = generate_updates(
    N, G, K, L, T, global_trajectories)

pi = np.ones(K) / K
psi = np.ones(L) / L
alpha = 0.1
rho = np.array([alpha, 1-alpha])

assignments = Assignments(pi, psi, rho, N, G, K, L, T)
m.weights = assignments.compute_weights()

In [ ]:
logger = train_mixsvgp(m, m_bar, assignments, x, y, n_iter, save_path)

UPDATING TRAJECTORIES


/home/karl/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4456517.074850
  Number of iterations: 43
  Number of functions evaluations: 49


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4456517.074850
  Number of iterations: 43
  Number of functions evaluations: 49


UPDATING HYPERPARAMETERS
UPDATING ASSIGNMENTS
2.97593355783109
SAVING:  ../output/K2_L10
UPDATING TRAJECTORIES
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4403082.511455
  Number of iterations: 28
  Number of functions evaluations: 30


...

UPDATING HYPERPARAMETERS
UPDATING ASSIGNMENTS
0.007677836313198032
SAVING:  ../output/K2_L10
UPDATING TRAJECTORIES
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3445794.286351
  Number of iterations: 22
  Number of functions evaluations: 25


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3445794.286351
  Number of iterations: 22
  Number of functions evaluations: 25


UPDATING HYPERPARAMETERS
UPDATING ASSIGNMENTS
0.009259077012134116
SAVING:  ../output/K2_L10
UPDATING TRAJECTORIES
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3448440.954321
  Number of iterations: 8
  Number of functions evaluations: 10


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 3448440.954321
  Number of iterations: 8
  Number of functions evaluations: 10


UPDATING HYPERPARAMETERS


In [5]:
assignments.Phi.sum(0)

array([ 7., 12.])

In [6]:
assignments.Lambda.sum(0)

array([ 530.65928622,  763.19587864,  765.14014561, 1339.51109474,
        259.02414903,  425.27651411,  594.36662502,  459.70937527,
        599.19028085,  462.7032167 ,  399.83920273,  781.74631338,
        365.32436209,  540.61914478,  617.26272761,  803.47962113,
       2910.0691777 , 1970.33501879,  847.21004767,  884.33781794])